# Sorting-Answer 

#### By Dae Hyeun (Isaac) Cheong 

This document has a code to sort a free response question into 5 categories: 
* **C**: Correct answer, but still need to be checked by human.
* **I**: Incorrect answer, but still need to be checked by human. 
* **H**: Answers that can't be classified using this algorithm so human needs to correct. 

In [1]:
library(knitr)
library(dplyr)
library(readxl)
library(stringr)
raw_data <- read_excel("Items 28 and 30-2.xls")
translated_pc <- read_excel("translated_pc.xlsx")


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



#### Methodology

To explain the methodology on how to sort an answer, I will use the question about water bubbles, which coresspond to question 4 of the science assessment. To the original dataset, the answer to the question 4 correspond to the column name "pc"

<img src="scoringrubric.png" alt="rubric" width="700"/>

Here, I will explain the psedocode of the following method.

1. We will first start with identifying MC (Most likely to be a correct answer). We consider the answer as most likely to be a correct answer when the answer contain at least two key words either from Code 10 and Code 11. We will make the two groups of key words (What and How), so the answer get classified into MC when answer has at least one word from each group.
    * For example, code 10's What group will be {Bubbles, Gas , They , etc...} and How group will be {light, less dense, not heavy, etc...}. If the student wrote "Bubbles float because air inside the bubble is lighter than water", the algorithm will classify the answer to MC group. 

2. We repeat the Step 1 for every Code of correct answer to finish the sorting to MC category.  

3. Now, out of all non-MC answer, we will identify CC (Likely to be a correct answer). If we look at the answer key, we can identify some important key words that the word alone makes the answer correct. When the answer contain at least one of this key words, then we identify the answer to category CC. 
    * For example, some key words for the bubble problem will be {less dense, not heavy, oxygen, carbon dioxide, light}. If the student wrote "The thing inside is less dense than water", this will not be classified into MC but CC.  
    
4. Then we repeat the STEP 3 to identify LI (Likely to be a incorrect answer). 

5. Any blank answer or the answer that contains {don't know, no idea}, go to category I. Rest of them go to NC (Not classified)


In [42]:
our_interest <- select(raw_data, CODECEN, CUADERNILLO, FORMA, PC)
final_df <- cbind(our_interest, translated_pc)
head(str_detect(final_df[ ,5], "(BECAUSE THE)\\s"), 10)

[1]  TRUE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE

In [48]:
final_df[-c(1,2,3), ]

,CODECEN,CUADERNILLO,FORMA,PC,pc
4,141100009,200646,B,PORQUE ESAS BURVUJAS SE EXPANDEN,BECAUSE THOSE BURVETS EXPAND
5,141100009,200644,B,PORQUE CUANDO UNO SOPLA SE ACEN BURBUJAS Y SE VAN A LAS NUBES Y SE ASE AGUA.,"BECAUSE WHEN ONE BLOWS, BUBBLES BECOME AND THEY GO TO THE CLOUDS AND WATER IS GROWN."
6,141100009,200647,A,ESTO SUCEDE DEBIDO A QUE EL AIRE NO PUEDE ESTAR O QUE DARSE EN EL AGU LO CUAL SE CONVIERTE EN BURBUJAS Y HACI LLEGA A LA SUPERFICIE,"THIS HAPPENS BECAUSE THE AIR CANNOT BE IN THE WATER, WHICH BECOMES BUBBLES AND REACHES THE SURFACE."
7,141100009,200169,A,LAS BURBUJAS SUBEN EN EL AGUA PORQUE ES AIRE Y EL AIRE DENTRO DE EL AGUA SUBE ASIA ARRIBA.,THE BUBBLES RISE IN THE WATER BECAUSE IT IS AIR AND THE AIR WITHIN THE WATER RISES ASIA UP.
8,141100009,200170,B,CUANDO SOOPLO DENTRO DE UN VASO DE AGUA CON UNA PAJILLA SE ASEN BURBUJAS POR EL AIRE.,"WHEN I BLOW INTO A GLASS OF WATER THROUGH A STRAW, BUBBLES ARE BROUGHT THROUGH THE AIR."
9,141100009,200171,A,LAS BURBUJAS SUBEN POR QUE UNO SOPLA DENTRO DEL AGUA Y LA FUERZA CON LA QUE UNA PERSONA SOPLA IMPULSA A LAS BURBUJAS HACIA ARRIBA.,BUBBLES RISE BECAUSE ONE BLOWS INTO THE WATER AND THE FORCE WITH WHICH A PERSON BLOWS PROPELS THE BUBBLES UP.
10,141100009,200172,B,PORQUE LAS BURBUJAS NO PESAN NADA Y ASI ELLAS SUBEN A LA SUPERFICIE.,BECAUSE BUBBLES WEIGH NOTHING AND SO THEY RISE TO THE SURFACE.
11,141100009,200173,A,POR QUE TIRAMOS AIRE Y SOPLAMOS CON NUETRA BOCA,WHY WE THROW AIR AND BLOW WITH OUR MOUTH
12,141100009,200174,B,PORQUE EL MISMO AIRE DE LA PERSONA ASE QUE EL AGUA AGA VURBUJAS,BECAUSE THE SAME AIR OF THE PERSON MAKES THE WATER MAKE VURBUJAS
13,130800004,200696,B,PORQUE SIGUEN EL AIRE,BECAUSE THEY FOLLOW THE AIR


In [14]:
## This function detect the answer that is most likely to be correct. 
## Take input as dataframe (translated text has to be a 5th column)
## Returns a another dataframe with only MC answer with another column labeled with "C" (Correct)
## What_words and How_words must be in Regex expression 

What_Words = "( AIR |BUBBLES|BUBBLE|THEY|OXYGEN| GAS | CARBON |CARBON DIOXIDE)"
How_Words = "( LIGHT |LESS DENSE|NOT HEAVY|LIGHTER|FLOAT|BLOWING| BLOW | BLOWS )"

detecting_MC <- function(x) {
    vector_of_answer <- x[ ,5]
    all_MC_answer <- str_detect(final_df[ ,5], What_Words) * str_detect(final_df[ ,5], How_Words)
    return(all_MC_answer) 
}


In [ ]:
## This function detects the row with completely wrong answer 
## Wrong answer = "Don't Know" + "NA" 
## Returns the dataframe with Wrong answer: This will be the "I" (incorrect) group

detecting_I <- function(df) {
    
}


In [46]:
## This function takes a key words and dataframe, 
## Returns the dataframe with keywords
## It narrows down the entire dataset to smaller subsets. 

narrower_w_keywords <- function(df, keywords) {
    vector_of_answer <- df[ ,5] 
    rows_w_keywords <- str_detect(vector_of_answer, keywords)
    rows_wo_keywords <- str_detect(vector_of_answer, keywords, negate = TRUE)
    return(df[rows_w_keywords, ])
    }

narrower_wo_keywords <- function(df, keywords) {
    vector_of_answer <- df[ ,5] 
    rows_wo_keywords <- str_detect(vector_of_answer, keywords, negate = TRUE)
    return(df[rows_wo_keywords, ])
    }

narrower_w_keywords(final_df, "BECAUSE THE\\s")

,CODECEN,CUADERNILLO,FORMA,PC,pc
1,141100009,200645,A,PORQUE LAS BURBUJAS ESTAN LLENAS DE AIRE OCIGENO Y NO DURAN DENTRO DEL AGUA POR ESO.,BECAUSE THE BUBBLES ARE FILLED WITH OXYGEN AIR AND DO NOT LAST IN THE WATER FOR THAT REASON.
3,141100009,200643,A,PORQUE LAS BURBUAS SALEN Y SALEN DEL POPOTE Y COMO NO TIENEN GRAVEDAD SUBEN Y SUBEN FLOTANDO.,BECAUSE THE BUBBLES COME AND COME OUT OF THE STRAW AND AS THEY HAVE NO GRAVITY THEY FLOAT UP AND UP.
6,141100009,200647,A,ESTO SUCEDE DEBIDO A QUE EL AIRE NO PUEDE ESTAR O QUE DARSE EN EL AGU LO CUAL SE CONVIERTE EN BURBUJAS Y HACI LLEGA A LA SUPERFICIE,"THIS HAPPENS BECAUSE THE AIR CANNOT BE IN THE WATER, WHICH BECOMES BUBBLES AND REACHES THE SURFACE."
12,141100009,200174,B,PORQUE EL MISMO AIRE DE LA PERSONA ASE QUE EL AGUA AGA VURBUJAS,BECAUSE THE SAME AIR OF THE PERSON MAKES THE WATER MAKE VURBUJAS
19,130800004,200207,A,PORQUE EL AGUA FLUYE Y LAS ELEBA A LA SURPERFICIE.,BECAUSE THE WATER FLOWS AND LIFTS THEM TO THE SURFACE.
22,130800004,200210,B,PORQUE LA PAJILLA LLEVA MOCHO AIRE CUANDO UNO SOPLA.,BECAUSE THE STRAW CARRIES A LOT OF AIR WHEN YOU BLOW.
25,132800001,200686,B,PORQUE LA PAJIA HASE VIENTO ENTONCE SE HACEN BURBUJAS,BECAUSE THE STRAW GETS WIND THEN THEY MAKE BUBBLES
32,132800001,200195,A,"SUBEN POR QUE LA FUERZA DEL SOPLIDO ASE FORMA, BURBUJAS LUEGO LA MISMA FUERZA LAS LEBANTA ASIALA SUPERFICIE.","THEY GO UP BECAUSE THE FORCE OF THE BLOWING ARE FORMED, BUBBLES THEN THE SAME FORCE RAISES THEM ASIALA THE SURFACE."
41,132800001,200691,A,PORQUE EL AIRE QUE EXPULSAS EN LA PAJILLA ESTA BAJO EL AGUA.,BECAUSE THE AIR YOU EXPELL IN THE STRAW IS UNDER WATER.
44,132800001,200682,B,PORQUE LA FUERZA DEL AIRE QUE SOPLA CON LA PAJILLA HACE QUE SUBAN BURBUJAS.,BECAUSE THE FORCE OF THE AIR BLOWING THROUGH THE STRAW CAUSES BUBBLES TO RISE.


In [21]:

Correct_answer = "(BLOWS AIR|BLOWS OXYGEN|LESS DENSE|NOT HEAVY|OXYGEN|AIR RISES|FLOAT| UP | SURFACE)"
detecting_MC <- function(x) {
    vector_of_answer <- x[ ,5]
    all_MC_answer <- str_detect(final_df[ ,5], Correct_answer)
    return(all_MC_answer) 
}

In [22]:
head(detecting_MC(final_df), 20)

[1]  TRUE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE